In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import SimpleRNN, CuDNNLSTM, GRU, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam, SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
dataset = pd.read_csv('data/train.csv')
dataset.head()
dataset.shape

(20800, 5)

In [3]:
nan_indexes = dataset.loc[pd.isna(dataset["text"]), :].index
dataset = dataset.drop(nan_indexes)
dataset.shape

(20761, 5)

In [4]:
print(142 in nan_indexes)

True


In [4]:
X = dataset.text
y = dataset.label

In [6]:
X[143]

'  U.N. Secretary General Complains That The ‘Masses Have Rejected Globalism’ In Favor Of Nationalism Antonio Guterres, elected in October to take over as U.N. secretary general next year, told a conference in his native Lisbon that this trend had undermined the willingness to receive refugees in Europe this year. He said the world must re-establish international protection for refugees coming from war zones such as Syria, but it would not be easy as developed countries were turning to nationalist agendas.   22, 2016 The incoming head of the United Nations warned on Tuesday that ‘losers of globalization’ in rich countries have felt ignored by establishment politicians, prompting them to turn to nationalist agendas, as in the U.S. election and Brexit referendum. “Therefore wait ye upon me, saith the LORD, until the day that I rise up to the prey: for my determination is to gather the nations, that I may assemble the kingdoms, to pour upon them mine indignation, even all my fierce anger:

In [7]:
mean_article_length = 0

for i in range(20761):
    if i in nan_indexes:
        continue
    mean_article_length += len(X[i].split(' '))

mean_article_length /= 20761

mean_article_length

773.6980395934685

In [5]:
max_words = 3000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [16]:
def LSTM_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,75,input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [10]:
from keras.callbacks import ModelCheckpoint
import keras as K

def train_model(model_id, model, optimizer, epochs, X_train, y_train, validation_split, batch_size):

    model.summary()
    
    checkpoint = ModelCheckpoint('NNs/model-{epoch:03d}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    tb_callback = K.callbacks.TensorBoard(log_dir='./logs/nn_' + str(model_id))

    hist = model.fit(X_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size, callbacks=[tb_callback, checkpoint])
    
    return hist

#hist = train_model(...)
#best_epoch = max(hist.history['val_loss'])



In [ ]:
model = LSTM_network()

train_model('LSTM 1 layer 75 words', model, Adam(), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1000, 75)          225000    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 96)                66432     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 97        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 291,529
Trainable params: 291,529
Non-trainable params: 0
_________________________________________________________________
Train on 16608 samples, validate on 4153 samples
Epoch 1/30
16608/16608 [==============================] - 8s 474us/step - loss: 0.4087 

In [ ]:
model.fit(sequences_matrix,y,batch_size=128,epochs=20,
          validation_split=0.2)


In [12]:
def RNN_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = SimpleRNN(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [16]:
model = RNN_network()

train_model('rnn', model, SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 64)                7360      
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [13]:
def GRU_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = GRU(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [14]:
model = GRU_network()

train_model('GRU', model, Adam(), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                22080     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [20]:
def LSTM2_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(78)(layer)
    layer = Dense(128,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [21]:
model = LSTM2_network()

train_model('LSTM2', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 78)                40248     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               10112     
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 129       
__________

In [22]:
max_words = 1500
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [23]:
def LSTM3_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [24]:
model = LSTM3_network()

train_model('LSTM3', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 1000, 50)          75000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [26]:
def LSTM4_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,25,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [27]:
model = LSTM4_network()

train_model('LSTM4', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 1000, 25)          37500     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                23040     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
_________________________________________________________________
activation_17 (Activation)   (None, 1)                 0         
Total params: 60,605
Trainable params: 60,605
Non-trainable params: 0
_________________________________________________________________
Train on 16608 samples, validate on 4153 samples
Epoch 1/15
16608/16608 [==============================] - 53s 3ms/step - loss: 0.4125 - a